In [1]:
import pickle
from bokeh.io import output_file, export
import numpy as np
import functions #mon fichier source

### Parameters

In [2]:
parameters = {
    "dataset": "cabspotting",# OPTIONS ARE: cabspotting geolife
    "user": 'oilrag',# Examples: abboip oilrag 51 90 (For geolife, an int index has to be specified, corresponding to the user index)
    "trajectory_index": 0, #Only for geolife, corresponds to the trajectory indices for each user
    "output_dir": 'output/sol', #output dir where we store the data (Data_real/ Data_obf)
    "save_file": 'sol_04112022_H30', #folder where we store solutions / data
    "epsilon": 300, #FLAIR's parameter (in meters)
    "horizon": 6,   #Prediction's horizon h (We'll thus predict the h next positions)
    "Tmin" : 0, # Studied time interval
    "Tmax" : 5000, 
    "sampling_time": 30, # Harmonisation period
    "POI_duration": 15, #Défini en minutes. On aura ensuite nbuf = POI_duration * 60 / sampling_time
    "privacy_limit": 500, #For the statistics in the pdf. We will study when we are below this limit.
    "images1": [ #Contains the images that are going to be on the sum-up pdf
        "xCoordinates.png",
        "yCoordinates.png",
        "obfuscated_traj.png"
    ],
    "images2": [
        "plot_util.png",
        "plot_priv.png"
    ]
}

### FLAIR p mpc-H

In [3]:
def create_pdf(parameters):
    data = functions.read_data(parameters)
    data = functions.process_data(parameters, data)
    xs, ys, bs, us, J, F, ts, util = functions.data_save(data, parameters)
    # On peut commencer par tracer les graphes spécifiques à la trajectoire en question
    Data_real = pickle.load( open( parameters['save_file']+"/Data_real.pkl", "rb" ))
    Data_obf = pickle.load( open( parameters['save_file']+"/Data_obf.pkl", "rb" ))
    functions.plot_traj(xs, ys, ts, "traj")
    
    #On lance mtn l'algo d'optimisation et on plot la privacy
    data_mpc = functions.solve_mpc(xs, ys, bs, us, parameters['horizon'],int(parameters['POI_duration']* 60 / parameters['sampling_time']), util, J, F)
    data_mpc_FLAIR = functions.solve_mpc_FLAIR(xs, ys, bs, us, parameters['sampling_time'], parameters['horizon'], int(parameters['POI_duration']* 60 / parameters['sampling_time']), util, parameters['epsilon'], J, F)
    functions.plot_privacy(Data_real, Data_obf, data_mpc, data_mpc_FLAIR, parameters['horizon'])
    #On plot l'utilité
    functions.plot_utility(Data_real, Data_obf, data_mpc, data_mpc_FLAIR, parameters['horizon'])
    functions.plot_whole_traj(xs, ys, parameters['sampling_time'], "obfuscated_traj", data_mpc_FLAIR, Data_obf, data_mpc, parameters['horizon'])
    # Stats
    privacy_limit = parameters['privacy_limit']
    statistics = functions.calculate_percentages(np.array(Data_real['priv'][:-parameters['horizon']]), np.array(Data_obf['priv'][:-parameters['horizon']]), np.array(data_mpc.priv), np.array(data_mpc_FLAIR.priv), privacy_limit)
    functions.pdf(parameters, statistics)
    return(data_mpc, data_mpc_FLAIR, statistics)
    

In [ ]:
data_mpc, data_mpc_FLAIR, statistics = create_pdf(parameters) 

In [ ]:
statistics


(81.16883116883116,
 17.532467532467532,
 87.66233766233766,
 11.03896103896104,
 24.025974025974026,
 20.77922077922078,
 12.987012987012985,
 12.987012987012985)

In [ ]:
#Pour différentes valeurs d'horizon
horizons = [1,3,5,10,15]
S = []
for h in horizons:
    parameters['horizon'] = h
    statistics = create_pdf(parameters)
    S.append(statistics)


### Evaluating FLAIR as a predictor

In [ ]:
#old code, wrong


# # On se contentera de la même trajectoire
# data = functions.read_data(parameters)
# data = functions.process_data(parameters, data)
# xs, ys, bs, us, J, F, ts, util = functions.data_save(data, parameters)
# xr = data['x'][(data['elapsedtime'] >= 0) & (data['elapsedtime'] < 5000)]
# # On peut commencer par tracer les graphes spécifiques à la trajectoire en question
# Data_real = pickle.load( open( parameters['save_file']+"/Data_real.pkl", "rb" ))
# Data_obf = pickle.load( open( parameters['save_file']+"/Data_obf.pkl", "rb" ))
# current_FLAIR_model_x, current_FLAIR_model_y = (0,0,0), (0,0,0)
# model_defined_x = False
# model_defined_y = False
# horizon, epsilon = parameters['horizon'], parameters['epsilon']
# Xf_FLAIR, Tx_FLAIR, X_FLAIR, Ax_FLAIR = [], [], [], []
# for tk in range (len(xs)- horizon):
#     if bs[tk] != 0:
#         Xf, model_defined_x, current_FLAIR_model_x = functions.predict_xf(current_FLAIR_model_x, tk, xs, horizon, model_defined_x, epsilon, bs, ts)
#         if tk!=0:
#             Ax_FLAIR.append(current_FLAIR_model_x[0])
#             Tx_FLAIR.append(tk*30)
#             X_FLAIR.append(current_FLAIR_model_x[3])
#         Xf_FLAIR.append(Xf)
#         Xf = Xf[::].tolist()


#         #à changer
#         #Yf = ys[tk:tk+horizon]
#         Yf, model_defined_y, current_FLAIR_model_y = functions.predict_xf(current_FLAIR_model_y, tk, ys, horizon, model_defined_y, epsilon, bs, ts)
#         Yf = Yf[::].tolist()

0 60
0 60
0 180
0 180
0 210
180 210
0 300
180 300
300 360
300 360
300 390
300 390
300 450
300 450
300 540
450 540
540 570
450 570
540 660
450 660
540 690
660 690
690 750
660 750
690 810
660 810
690 900
660 900
690 960
660 960
690 1020
960 1020
1020 1080
960 1080
1020 1140
960 1140
1020 1200
1140 1200
1020 1260
1140 1260
1020 1320
1140 1320
1320 1380
1140 1380
1320 1440
1140 1440
1320 1500
1140 1500
1500 1560
1500 1560
1500 1620
1500 1620
1500 1680
1500 1680
1500 1740
1500 1740
1740 1800
1500 1800
1740 1860
1500 1860
1740 1920
1500 1920
1920 1980
1500 1980
1920 2040
1500 2040
1920 2100
1500 2100
1920 2160
1500 2160
1920 2220
1500 2220
1920 2250
1500 2250
2250 2310
1500 2310
2250 2370
1500 2370
2250 2430
1500 2430
2250 2490
1500 2490
2250 2550
1500 2550
2250 2610
1500 2610
2610 2670
2610 2670
2610 2730
2610 2730
2730 2790
2730 2790
2730 2850
2730 2850
2730 2910
2730 2910
2910 2970
2730 2970
2910 3030
2730 3030
2910 3090
2730 3090
3090 3150
3090 3150
3090 3180
3090 3180
3180 3240
3180 324

In [11]:
#FLAIR as a predictor
#This will be for a horizon of just 1, this is just to get an idea 
#This time we won't go through the harmonisation of the trajectory
data = functions.read_data(parameters)
data = functions.process_data(parameters, data)
data = data[(data['elapsedtime'] >= parameters['Tmin']) & (data['elapsedtime'] < parameters['Tmax'])]
current_FLAIR_model_x = (0, data['elapsedtime'][0], data['x'][0])
current_FLAIR_model_y = (0, data['elapsedtime'][0], data['y'][0])
x0, y0, x1, y1, t0, t1 = data['x'][0], data['y'][0], data['x'][1], data['y'][1], 0, data['elapsedtime'][1]
t0y, t1y = 0, data['elapsedtime'][1]
FLAIR_prediction_x = [0,0]
FLAIR_prediction_y = [0,0]
A0x = (x1 - x0)/ (t1 - t0)
A0y = (y1 - y0)/ (t1 - t0)
A_min_x = (x1-parameters['epsilon']-x0)/(t1-t0 +1e-12)
A_max_x = (x1+parameters['epsilon']-x0)/(t1-t0 +1e-12)
A_min_y = (y1-parameters['epsilon']-y0)/(t1y-t0y +1e-12)
A_max_y = (y1+parameters['epsilon']-y0)/(t1y-t0y +1e-12)
for i in range(2,len(data)):
    x, y, t = data['x'][i], data['y'][i], data['elapsedtime'][i]
    FLAIR_prediction_x.append(x0 + (data['elapsedtime'][i]-t0)*A0x)
    FLAIR_prediction_y.append(y0 + (data['elapsedtime'][i]-t0y)*A0y)
    # differences_x = np.abs(np.array(FLAIR_prediction_x) - data['x'])
    # mean_distance_x = np.mean(differences_x)
    x0, t0, x1, t1, A0x, A_min_x, A_max_x = functions.FLAIR_insert(x0, x1, x, t0, t1, t, A0x, A_min_x, A_max_x, parameters['epsilon'])
    y0, t0y, y1, t1y, A0y, A_min_y, A_max_y = functions.FLAIR_insert(y0, y1, y, t0y, t1y, t, A0y, A_min_y, A_max_y, parameters['epsilon'])
    



In [ ]:
differences_x = np.abs(np.array(FLAIR_prediction_x) - data['x'])
mean_distance_x = np.mean(differences_x)
print('Mean error (horizon = 1): \n')
print(mean_distance_x)


Mean error: 

207.2860800515195
